In [5]:
import os
import xml.etree.ElementTree as ET
import json
import random

In [6]:
val_dir = './datasets/kaist-rgbt/val.txt'

txt_dir = "./datasets/kaist-rgbt/train/labels"
xml_dir = "./datasets/kaist-rgbt/train/labels-xml"

json_dir = './utils/eval/KAIST_annotation2.json'

In [7]:
annotation = {
    "info": {
        "dataset": "KAIST Multispectral Pedestrian Benchmark",
        "url": "https://soonminhwang.github.io/rgbt-ped-detection/",
        "related_project_url": "http://multispectral.kaist.ac.kr",
        "publish": "CVPR 2015"
    },
    "info_improved": {
        "sanitized_annotation": {
            "publish": "BMVC 2018",
            "url": "https://li-chengyang.github.io/home/MSDS-RCNN/",
            "target": "files in train-all-02.txt (set00-set05)"
        },
        "improved_annotation": {
            "url": "https://github.com/denny1108/multispectral-pedestrian-py-faster-rcnn",
            "publish": "BMVC 2016",
            "target": "files in test-all-20.txt (set06-set11)"
        }
    },
    "images": [],
    "annotations": [],
    "categories": [
        {
            "id": 0,
            "name": "person"
        },
        {
            "id": 1,
            "name": "cyclist"
        },
        {
            "id": 2,
            "name": "people"
        },
        {
            "id": 3,
            "name": "person?"
        }
    ]
}

In [8]:
with open(val_dir, 'r') as file:
    val_names = file.readlines()

xml_files = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]
txt_files = [f for f in os.listdir(txt_dir) if f.endswith('.txt')]

image_id = 0
annotation_id = 0

for val_name in val_names:
    for xml_file, txt_file in zip(sorted(xml_files), sorted(txt_files)):
        tree = ET.parse(os.path.join(xml_dir, xml_file))
        root = tree.getroot()
        im_name = xml_file.replace('.xml', '')

        if val_name.find(im_name) != -1:
            width = int(root.find("size/width").text)
            height = int(root.find("size/height").text)
            root.find("filename")

            annotation["images"].append({
                "id": image_id,
                "im_name": im_name,
                "height": height,
                "width": width
            })

            with open(os.path.join(txt_dir, txt_file), 'r') as file:
                lines = file.readlines()
                for line in lines:
                    if line.strip(): 
                        parts = line.strip().split()
                        class_id = int(parts[0])
                        x_center = float(parts[1]) * 640
                        x_center = float(f"{x_center:.3f}")
                        y_center = float(parts[2]) * 512
                        y_center = float(f"{y_center:.3f}")
                        width = float(parts[3]) * 640
                        width = float(f"{width:.3f}")
                        height = float(parts[4]) * 512
                        height = float(f"{height:.3f}")
                        ignore = int(parts[5])
                        x_min = x_center - width / 2
                        y_min = y_center - height / 2
                        

                        annotation["annotations"].append({
                            "id": annotation_id,
                            "image_id": image_id,
                            "category_id": class_id,
                            "bbox": [x_min, y_min, width, height],
                            "height": height,
                            "occlusion": 0, 
                            "ignore": ignore
                        })
                        annotation_id += 1

            image_id += 1

        else:
            pass

with open(json_dir, 'w') as json_file:
    json.dump(annotation, json_file, indent=4)

print("complete")

complete


In [ ]:
print(annotation)